In [1]:
def read_cells(filename):

  hard_macro_info={}
  std_info={}
  hard_macro_name=[]
  std_name=[]

  componentstart=0

  #lef 파일로부터 component들의 정보 읽어오기

  with open(filename + '.lef') as n:

    for num, line in enumerate(n):

      if '#' in line:
        continue
      if 'MACRO' in line: #새로운 component 종류 등장
        componentstart = 1
        ismacro = 0
        data = line.split()
        component_name = data[1]
      elif 'CLASS' in line: #component가 macro인지 std인지 분류
        data = line.split()
        if(data[1]=='BLOCK'):
          ismacro = 1 #hard macro
        else:
          ismacro = 0


      elif 'BY' in line:
        if(componentstart):
          data = line.split()
          width = data[1]
          height = data[3]

          if(ismacro):
            hard_macro_info[component_name] = {'width' : float(width), 'height' : float(height)}
            hard_macro_name.append(component_name)
          else:
            try:
              std_info[component_name] = {'width' : float(width), 'height' : float(height)}
            except ValueError:
              std_info[component_name] = {'width' : float(data[2]), 'height' : float(data[4])}
            std_name.append(component_name)
      else:
        continue

  hard_macros = {}
  hard_macro_indices=[]
  stds = {}
  std_indices=[]

  #def 파일로부터 cell들의 정보 읽어오기
  with open(filename + '.def') as n:
    for num, line in enumerate(n):
      if '- inst' in line: #새로운 cell 등장
        
        data = line.split()
        cell_num = data[1]
        cell_num=int(cell_num[4:])

        cell_name = data[2] 
        cell_x = data[6]
        cell_y = data[7]
        cell_type = data[9]

        #make std and macro cell list
        if cell_name in std_name:
            stds[cell_num-1] = std_info[cell_name]
            std_indices.append(cell_num-1) #def 파일 상의 실제 cell 번호

        elif cell_name in hard_macro_name: #macro cell list
            hard_macros[cell_num-1] = hard_macro_info[cell_name]
            hard_macro_indices.append(cell_num-1) #def 파일 상의 실제 cell 번호
      else:
        continue

  return hard_macros, hard_macro_indices, stds, std_indices

In [2]:
def read_nets(filename, std_indices, hard_macro_indices):


  net_list = []
  net_list_for_hmetis = []



  with open(filename + '.def') as n:
    for num, line in enumerate(n):
      if 'POWER' in line:
        break
      if '- net' in line: #새로운 net 등장
        net_list.append([])
        net_list_for_hmetis.append([])

      elif '( inst' in line:
        data = line.split()
        for i in data:
          if 'inst' in i:
            cell_num = int(i[4:])-1

            if cell_num in std_indices: #std cell만 net에 포함시킨다
              net_list[-1].append(cell_num) #cellnum으로
              net_list_for_hmetis[-1].append(std_indices.index(cell_num)+1) #실제 cell num 대신 cell num이 std_indices에 들어있는 순서 대입
            else:
              net_list[-1].append(cell_num)


  return net_list, net_list_for_hmetis

In [3]:
def softmacro(partition_num, std_indices,stds,hard_macros):

  soft_macro_element = {}
  for part in range(len(hard_macros), len(hard_macros)+partition_num):
    soft_macro_element[part] = []
  soft_macro_indices = [i for i in range(len(hard_macros), len(hard_macros)+partition_num)]
  soft_macros = {}
  std_softmacro_map = {}
  
  with open('./netlist/HGraphFile.hgr.part.'+str(partition_num)) as n:
    for line, num in enumerate(n):
      soft_macro_element[int(num)+len(hard_macros)].append(std_indices[line]) #soft macro 각각에 포함된 std cell의 번호
      std_softmacro_map[std_indices[line]] = int(num)+len(hard_macros)



  for i in range(len(hard_macros),len(hard_macros)+partition_num): #i번째 soft macro
    height = 1.71 * len(soft_macro_element[i])
    width = 0
    for j in soft_macro_element[i]: #j는 soft macro에 포함된 std cell의 번호
      #print(std_indices.index(j))
      width += stds[j]['width']
    
    from math import sqrt
    soft_macros[i] = {'height' : sqrt(height), 'width': sqrt(width)}
  
  return soft_macro_element, soft_macro_indices, soft_macros, std_softmacro_map

In [4]:
def hmetis_dataset(net_list, stds):
  
  #macro 만으로 이루어져 있어 빈 netlist 원소 개수
  empty=0
  for n in net_list:
    if len(n)==0 or len(n)==1:
      empty += 1

  
  f=open("./netlist/HGraphFile.txt", 'w')
  #write the information of graph at the top
  info_data="%d %d\n" % (len(net_list)-empty, len(stds))
  f.write(info_data) 

  for i in net_list:
    data = ""
    for j in i:
      data = data + "%d " % j
    data = data + "\n"

    if data!="\n":
      f.write(data)

  f.close()

def make_HGraphFile(filename):
  hard_macros, hard_macro_indices, stds, std_indices=read_cells(filename)
  net_list,net_list_for_hmetis,macro_net_list=read_nets(filename,std_indices,hard_macro_indices)
  #cell_number=len(cells)

  hmetis_dataset(net_list_for_hmetis, stds)

In [5]:
def parsing(filename,partition_number=100):

  hard_macros, hard_macro_indices, stds, std_indices=read_cells(filename)
  net_list,net_list_for_hmetis=read_nets(filename,std_indices,hard_macro_indices)
  #cell_number=len(cells)
  
  soft_macro_element, soft_macro_indices, soft_macros, std_softmacro_map = softmacro(partition_number,std_indices,stds,hard_macros)

  macros = hard_macros|soft_macros

  #make macro net list
  #hard macro만 포함된 net list에 만들어진 soft macro의 net 정보를 추가한다
  for i in hard_macro_indices:
    std_softmacro_map[i] = i

  macro_net_list = [[] for i in range(len(net_list))]

  for net_idx, net in enumerate(net_list):
    for cell in net:
      macro_net_list[net_idx].append(std_softmacro_map[cell])

  
  #make adjacancy matrix
  adjacency_matrix = [[0 for i in range(len(macros))] for j in range(len(macros))]
  for net in macro_net_list:
      for i in net:
          for j in net:
              adjacency_matrix[i][j] = 1 # += 1

  for i in range(len(adjacency_matrix)):
    adjacency_matrix[i][i] = 0

  #print(macro_net_list)


  return adjacency_matrix, hard_macros, soft_macros, macros, hard_macro_indices, soft_macro_indices, macro_net_list

In [6]:
filename = './netlist/ispd18_test3'
adjacency_matrix, macros, stds, cells, macro_indices, std_indices, macro_net_list = parsing(filename,partition_number=4)

In [7]:
print(macro_indices)

[3, 2, 1, 0]


In [8]:
import pickle

with open("./netlist/adjacency_matrix", "wb") as f:
    pickle.dump(adjacency_matrix, f)

with open("./netlist/cells", "wb") as f:
    pickle.dump(cells, f)

with open("./netlist/macro_indices", "wb") as f:
    pickle.dump(macro_indices, f)

with open("./netlist/std_indices", "wb") as f:
    pickle.dump(std_indices, f)